In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import plasma.functional as F

from plasma.meta.object_graph import ContextManager, Inputs
from typing import Callable

In [3]:
class Handler(F.AutoPipe):

    def __init__(self, name, age, test=5):
        super().__init__()
        self.name = name
        self.age = age
        self.test = test

In [4]:
class Flow(F.AutoPipe):

    def __init__(self, handler, bee) -> None:
        super().__init__()
        self.handler = handler

In [5]:
class Service(F.AutoPipe):

    def __init__(self, flow:Flow, handler:Callable[[int, int], str], test_factory:dict, check:int) -> None:
        super().__init__()
        print(test_factory)
        self.flow = flow
        self.handler = handler

In [6]:
class AnotherHandler(F.AutoPipe):
    
    def __init__(self, test_factory:dict, check:int, check2:int):
        super().__init__()
        
        self.test_factory = test_factory

In [7]:
manager =  ContextManager()
manager

In [8]:
service_manager = manager\
    .context('service_components')\
        .add_dependency('main', Service)\
        .add_dependency('another_handler', AnotherHandler)

service_manager

service_components
  |-> main
    |-> flow: Flow
    |-> handler: F[[int, int],str]
    |-> test_factory: dict
    |-> check: int
------------------------------------------
service_components
  |-> another_handler
    |-> test_factory
    |-> check
    |-> check2: int
------------------------------------------

In [ ]:
factory = service_manager.factory('test_factory')
factory[8, 6] = 'hehe'

service_manager

In [ ]:
service_manager.inputs('main', 'another_handler')

In [ ]:
package_manager = manager\
    .context('package')\
        .add_dependency('flow', Flow)\
        .add_dependency('handler', Handler)
package_manager           

In [ ]:
service_manager.link_name(package_manager)

In [ ]:
requirements = service_manager.inputs('main', 'another_handler')
requirements

In [ ]:
class TestInputs2:...


class TestInputs(Inputs):
    check:int
    check2:int
    package:TestInputs2


class TestInputs2(Inputs):
    age:int
    bee:str
    name:str

In [ ]:
inputs = TestInputs({'check': 8, 'check2': 12, 'package': {'age': 32, 'bee': 'buzzbuzz', 'name': 'tèo'}})
inputs 

In [ ]:
main, handler = inputs.init(service_manager, 'main', 'another_handler')

In [ ]:
main

In [ ]:
handler